In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
#!conda install pytorch torchvision -c pytorch
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/huggingface-bert/bert-base-multilingual-cased/config.json
/kaggle/input/huggingface-bert/bert-base-multilingual-cased/tokenizer.json
/kaggle/input/huggingface-bert/bert-base-multilingual-cased/tf_model.h5
/kaggle/input/huggingface-bert/bert-base-multilingual-cased/pytorch_model.bin
/kaggle/input/huggingface-bert/bert-base-multilingual-cased/modelcard.json
/kaggle/input/huggingface-bert/bert-base-multilingual-cased/vocab.txt
/kaggle/input/huggingface-bert/bert-large-uncased/config.json
/kaggle/input/huggingface-bert/bert-large-uncased/tokenizer.json
/kaggle/input/huggingface-bert/bert-large-uncased/tf_model.h5
/kaggle/input/huggingface-bert/bert-large-uncased/pytorch_model.bin
/kaggle/input/huggingface-bert/bert-large-uncased/modelcard.json
/kaggle/input/huggingface-bert/bert-large-uncased/vocab.txt
/kaggle/input/huggingface-bert/bert-large-uncased/whole-word-masking/._bert_config.json
/kaggle/input/huggingface-bert/bert-large-uncased/whole-word-masking/bert_config.json
/k

In [2]:
# necessary libraries
#import os
import random
import pandas as pd 
#import numpy as np 
import re
from spacy.tokens import Doc
import spacy
import torch
import torch.nn as nn
from spacy.tokenizer import Tokenizer
from tokenizers import BertWordPieceTokenizer
import urllib
from spacy.tokens import Doc
#from spacy.lang.en import English
from matplotlib import pyplot as plt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
#from tensorflow.keras.preprocessing.text import Tokenizer
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.optimizers import Adam
#from tensorflow.keras import regularizers
import tensorflow.keras.utils 
import tensorflow as tf
from transformers import AutoTokenizer, AutoModelForMaskedLM, BertTokenizer
import gensim
from gensim.test.utils import common_texts
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
w2v_model = Word2Vec(sentences = common_texts, vector_size = 100, window = 5, min_count = 1, workers = 4)
w2v_model.save("word2vec.model")

#os.makedirs('../word2vec')

MODEL_DIR = "/kaggle/input/huggingface-bert/"
#tokenizer = BertTokenizer(MODEL_DIR + "bert-large-uncased").encode()
#model = AutoModelForMaskedLM.from_pretrained(MODEL_DIR + "bert-large-uncased")
#model = Word2Vec(sentences, min_count=1)
#tokenizer = Word2Vec(sentences, min_count=1)
nlp = spacy.load('en_core_web_lg')

# loading the dataset
nlp_df = pd.read_csv(r'''../input/medium-articles/medium_articles.csv''')[:100]
nlp_df.head()
nlp_df_texts = nlp_df['text']
nlp_df_texts.head()

0    Photo by Josh Riemer on Unsplash\n\nMerry Chri...
1    Your Brain On Coronavirus\n\nA guide to the cu...
2    Mind Your Nose\n\nHow smell training can chang...
3    Passionate about the synergy between science a...
4    You’ve heard of him, haven’t you? Phineas Gage...
Name: text, dtype: object

In [3]:
#general unit tests
import doctest
import copy
import functools

def autotest(func):
    globs = copy.copy(globals())
    globs.update({func.__name__: func})
    doctest.run_docstring_examples(
        func, globs, verbose=True, name=func.__name__)
    return func

In [4]:
#Copied from the course- setting the random seed.
# @title Set random seed

# @markdown Executing `set_seed(seed=seed)` you are setting the seed

# For DL its critical to set the random seed so that students can have a
# baseline to compare their results to expected results.
# Read more here: https://pytorch.org/docs/stable/notes/randomness.html

# Call `set_seed` function in the exercises to ensure reproducibility.
import random
import torch

def set_seed(seed=None, seed_torch=True):
  """
  Function that controls randomness. NumPy and random modules must be imported.

  Args:
    seed : Integer
      A non-negative integer that defines the random state. Default is `None`.
    seed_torch : Boolean
      If `True` sets the random seed for pytorch tensors, so pytorch module
      must be imported. Default is `True`.

  Returns:
    Nothing.
  """
  if seed is None:
    seed = np.random.choice(2 ** 32)
  random.seed(seed)
  np.random.seed(seed)
  if seed_torch:
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

  print(f'Random seed {seed} has been set.')


# In case that `DataLoader` is used
def seed_worker(worker_id):
  """
  DataLoader will reseed workers following randomness in
  multi-process data loading algorithm.

  Args:
    worker_id: integer
      ID of subprocess to seed. 0 means that
      the data will be loaded in the main process
      Refer: https://pytorch.org/docs/stable/data.html#data-loading-randomness for more details

  Returns:
    Nothing
  """
  worker_seed = torch.initial_seed() % 2**32
  np.random.seed(worker_seed)
  random.seed(worker_seed)

In [5]:
doc = nlp("It's a simple sentence. Second simple sentence")
for token in doc:
    print(token)


It
's
a
simple
sentence
.
Second
simple
sentence


In [6]:
# @title Set device (GPU or CPU). Execute `set_device()`
# especially if torch modules used.

# Copied from the course materials.
# Inform the user if the notebook uses GPU or CPU.

def set_device():
  """
  Set the device. CUDA if available, CPU otherwise

  Args:
    None

  Returns:
    Nothing
  """
  device = "cuda" if torch.cuda.is_available() else "cpu"
  if device != "cuda":
    print("WARNING: For this notebook to perform best, "
        "if possible, in the menu under `Runtime` -> "
        "`Change runtime type.`  select `GPU` ")
  else:
    print("GPU is enabled in this notebook.")

  return device

In [7]:
SEED = 2021
set_seed(seed=SEED)
DEVICE = set_device()

Random seed 2021 has been set.
GPU is enabled in this notebook.


In [8]:
str = 'abc'
if not str[-1] == 'b':
    str = str + 'c'
print(str)

abcc


In [9]:
import tempfile

with tempfile.NamedTemporaryFile(prefix='gensim-model-', delete=False) as tmp:
    temporary_filepath = tmp.name
    w2v_model.save(temporary_filepath)
    #
    # The model is now safely stored in the filepath.
    # You can copy it to other machines, share it with others, etc.
    #
    # To load a saved model:
    #
    new_model = gensim.models.Word2Vec.load(temporary_filepath)

In [10]:
>>> '\x80'.encode().decode('windows-1252')

'Â€'

In [11]:
#data preprocessing and Tokenize
default_length = 100
#model = gensim.models.KeyedVectors.load_word2vec_format('/tmp/vectors.txt', binary=False)
# using gzipped/bz2 input works too, no need to unzip

@autotest
def get_sentences(article):
    '''
    Split the article into sentences.
    >>> get_sentences("First sentence. Second sentence.")
    ['First sentence.', 'Second sentence.']
    '''
    list_of_sentences = []
    for sentence in article.split(". "):
        if sentence:
            sentence_copy = sentence
            if not sentence_copy[-1] == '.':
                sentence_copy = sentence_copy + '.'
            list_of_sentences.append(sentence_copy)
#        if len(list_of_sentences) >= 25:
#            print(list_of_sentences[-1])
    return list_of_sentences

@autotest
def train_dictionary(list_of_sentences, autotest = 0):
    '''
    Adds words to the dictionary
    >>> train_dictionary([["First", "sentence", "sentence"]], autotest = 1)
    [-0.00950012  0.00956222 -0.00777076 -0.00264551 -0.00490641 -0.0049667
     -0.00802359 -0.00778358 -0.00455321 -0.00127536 -0.00510299  0.00614054
     -0.00951662 -0.0053071   0.00943715  0.00699133  0.00767581  0.00423474
      0.00050709 -0.00598114  0.00601878  0.00263503  0.00769943  0.00639384
      0.00794257  0.00865741 -0.00989576 -0.0067557   0.00133757  0.0064403
      0.00737381  0.00551698  0.00766162 -0.00512557  0.00658441 -0.00410837
     -0.00905534  0.00914168  0.0013314  -0.00275968 -0.00247784 -0.00422048
      0.00481234  0.00440022 -0.00265336 -0.00734188 -0.00356585 -0.00033662
      0.00609589 -0.00283734 -0.00012089  0.00087973 -0.00709565  0.002065
     -0.00143242  0.00280215  0.00484222 -0.00135202 -0.00278014  0.00773865
      0.0050456   0.00671352  0.00451564  0.00866715  0.00747497 -0.00108189
      0.00874764  0.00460172  0.00544063 -0.00138608 -0.00204132 -0.00442435
     -0.0085152   0.00303773  0.00888319  0.00891974 -0.00194236  0.00608616
      0.00377972 -0.00429597  0.00204292 -0.00543789  0.00820889  0.00543291
      0.00318443  0.00410257  0.00865715  0.00727203 -0.00083347 -0.00707277
      0.00838047  0.00723358  0.00173047 -0.00134749 -0.00589009 -0.00453309
      0.00864797 -0.00313511 -0.00633882  0.00987008]
    
    '''
    #total_examples=model.corpus_count, epochs=model.epochs
#    w2v_model = Word2Vec.load("word2vec.model")
    w2v_model = gensim.models.Word2Vec.load("/kaggle/working/word2vec.model")
    w2v_model.build_vocab(list_of_sentences, progress_per=10000, update = True)
    w2v_model.train(list_of_sentences, total_examples = w2v_model.corpus_count, epochs = w2v_model.epochs)
#    word_vectors = w2v_model.wv
#    word_vectors.save("word2vec.wordvectors")
    w2v_model.save("word2vec.model")

    # Load back with memory-mapping = read-only, shared across processes.
    wv = gensim.models.KeyedVectors.load("/kaggle/working/word2vec.model")
    if autotest:
        print(w2v_model.wv['sentence'])

@autotest
def tokenizer(sentences):
    '''
    Prepares final, vectorized token.
    >>> tokenizer(np.array(["computer", "computer", 0]))
    [array([-0.00515774, -0.00667028, -0.0077791 ,  0.00831315, -0.00198292,
           -0.00685696, -0.0041556 ,  0.00514562, -0.00286997, -0.00375075,
            0.0016219 , -0.0027771 , -0.00158482,  0.0010748 , -0.00297881,
            0.00852176,  0.00391207, -0.00996176,  0.00626142, -0.00675622,
            0.00076966,  0.00440552, -0.00510486, -0.00211128,  0.00809783,
           -0.00424503, -0.00763848,  0.00926061, -0.00215612, -0.00472081,
            0.00857329,  0.00428458,  0.0043261 ,  0.00928722, -0.00845554,
            0.00525685,  0.00203994,  0.0041895 ,  0.00169839,  0.00446543,
            0.00448759,  0.0061063 , -0.00320303, -0.00457706, -0.00042664,
            0.00253447, -0.00326412,  0.00605948,  0.00415534,  0.00776685,
            0.00257002,  0.00811904, -0.00138761,  0.00808028,  0.0037181 ,
           -0.00804967, -0.00393476, -0.0024726 ,  0.00489447, -0.00087241,
           -0.00283173,  0.00783599,  0.00932561, -0.0016154 , -0.00516075,
           -0.00470313, -0.00484746, -0.00960562,  0.00137242, -0.00422615,
            0.00252744,  0.00561612, -0.00406709, -0.00959937,  0.00154715,
           -0.00670207,  0.0024959 , -0.00378173,  0.00708048,  0.00064041,
            0.00356198, -0.00273993, -0.00171105,  0.00765502,  0.00140809,
           -0.00585215, -0.00783678,  0.00123304,  0.00645651,  0.00555797,
           -0.00897966,  0.00859466,  0.00404815,  0.00747178,  0.00974917,
           -0.0072917 , -0.00904259,  0.0058377 ,  0.00939395,  0.00350795],
          dtype=float32), array([-0.00515774, -0.00667028, -0.0077791 ,  0.00831315, -0.00198292,
           -0.00685696, -0.0041556 ,  0.00514562, -0.00286997, -0.00375075,
            0.0016219 , -0.0027771 , -0.00158482,  0.0010748 , -0.00297881,
            0.00852176,  0.00391207, -0.00996176,  0.00626142, -0.00675622,
            0.00076966,  0.00440552, -0.00510486, -0.00211128,  0.00809783,
           -0.00424503, -0.00763848,  0.00926061, -0.00215612, -0.00472081,
            0.00857329,  0.00428458,  0.0043261 ,  0.00928722, -0.00845554,
            0.00525685,  0.00203994,  0.0041895 ,  0.00169839,  0.00446543,
            0.00448759,  0.0061063 , -0.00320303, -0.00457706, -0.00042664,
            0.00253447, -0.00326412,  0.00605948,  0.00415534,  0.00776685,
            0.00257002,  0.00811904, -0.00138761,  0.00808028,  0.0037181 ,
           -0.00804967, -0.00393476, -0.0024726 ,  0.00489447, -0.00087241,
           -0.00283173,  0.00783599,  0.00932561, -0.0016154 , -0.00516075,
           -0.00470313, -0.00484746, -0.00960562,  0.00137242, -0.00422615,
            0.00252744,  0.00561612, -0.00406709, -0.00959937,  0.00154715,
           -0.00670207,  0.0024959 , -0.00378173,  0.00708048,  0.00064041,
            0.00356198, -0.00273993, -0.00171105,  0.00765502,  0.00140809,
           -0.00585215, -0.00783678,  0.00123304,  0.00645651,  0.00555797,
           -0.00897966,  0.00859466,  0.00404815,  0.00747178,  0.00974917,
           -0.0072917 , -0.00904259,  0.0058377 ,  0.00939395,  0.00350795],
          dtype=float32)]
    '''
    words_vector = []
    for word in sentences:
        if word in w2v_model.wv.key_to_index:
            vector = w2v_model.wv[word]
            words_vector.append(vector)
    return words_vector
            
@autotest
def padding(sentence, max_sentence_length = default_length, labels = 0):
#    print(sentence.shape[0] < max(default_length, max_sentence_length))
#    while sentence.shape[0] < max(default_length, max_sentence_length):
#        if labels:
#            sentence[-1] = 0
#        sentence = np.append(sentence, [0])
    while len(sentence) < max(default_length, max_sentence_length):
        if labels:
            sentence[-1] = 0
        sentence = np.append(sentence, [0])
#        print(sentence)
#    print(sentence)
    return sentence
        
@autotest
def sentence_token(sentence, vectorized = 1, labels = 0):
    '''
    Creating tokens for each sentence.
    >>> sentence_token("It's a simple sentence.", 0)
    array([It, 's, a, simple, sentence, ., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=object)
    '''
    if vectorized:
        sentence_token = []
    else:
        sentence_token = []
    doc = nlp(sentence)
    for token in doc:
#        if vectorized:
#            sentence_token = torch.cat((sentence_token, torch.tensor([[Tokenizer(token)]])), dim = 1)
#        else:
#            sentence_token = torch.cat((sentence_token, token), dim = 1)
        if vectorized:
            sentence_token.append([tokenizer(token)])
        else:
            sentence_token.append(token)
    
#    print(sentence_token.shape[0])
    sentence_token = padding(sentence_token, labels)
#    print(padding(sentence_token, training = 0 ))
#    print(sentence_token)
    return sentence_token
    
@autotest
def tokenize_article(article, vectorized = 1, labels = 0):
    '''
    Creates a list of lists of tokens of the article. The last sentence of the article retains a dot.
    >>> tokenize_article("First sentence. Second sentence.", 0)
    [array([First, sentence, ., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=object), array([Second, sentence, ., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=object)]
    '''
    sentence_token_list = []
    for sentence in get_sentences(article):
        sentence_token_list.append(sentence_token(sentence,vectorized, labels))
    return sentence_token_list

Finding tests in get_sentences


Trying:
    get_sentences("First sentence. Second sentence.")
Expecting:
    ['First sentence.', 'Second sentence.']
ok
Finding tests in train_dictionary
Trying:
    train_dictionary([["First", "sentence", "sentence"]], autotest = 1)
Expecting:
    [-0.00950012  0.00956222 -0.00777076 -0.00264551 -0.00490641 -0.0049667
     -0.00802359 -0.00778358 -0.00455321 -0.00127536 -0.00510299  0.00614054
     -0.00951662 -0.0053071   0.00943715  0.00699133  0.00767581  0.00423474
      0.00050709 -0.00598114  0.00601878  0.00263503  0.00769943  0.00639384
      0.00794257  0.00865741 -0.00989576 -0.0067557   0.00133757  0.0064403
      0.00737381  0.00551698  0.00766162 -0.00512557  0.00658441 -0.00410837
     -0.00905534  0.00914168  0.0013314  -0.00275968 -0.00247784 -0.00422048
      0.00481234  0.00440022 -0.00265336 -0.00734188 -0.00356585 -0.00033662
      0.00609589 -0.00283734 -0.00012089  0.00087973 -0.00709565  0.002065
     -0.00143242  0.00280215  0.00484222 -0.00135202 -0.00278014  

In [ ]:
dataset = nlp_df_texts
def create_vocab(dataset):
    '''
    Split into sentences -> create sentence tokens -> apply word2vec to sentence tokens of the form: [["str", "str", ..., "."], ["str", "str", ..., "."], ...]
    '''
    for article in dataset:
        list_of_sentences = tokenize_article(article, 0)
#        print(list_of_sentences)
        train_dictionary(list_of_sentences)
#    w2v_model = KeyedVectors.load("word2vec.model", mmap='r')
    w2v_model = gensim.models.Word2Vec.load("/kaggle/working/word2vec.model")
    print(w2v_model.wv['synergy'])
create_vocab(dataset)

In [ ]:
#RNN (LSTM)
dataset = nlp_df_texts
model = Word2Vec.load("word2vec.model")
class RNN_LSTM(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, n_layers, lr):
        super(RNN_LSTM, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.num_layers = n_layers
        self.lr = lr
        self.training_dataset, self.labels_dataset = self.load_all_articles(dataset)
        self.modnlp = nn.Sequential(
            nn.LSTM(input_size = embedding_dim, hidden_size = self.hidden_dim, num_layers = self.num_layers, batch_first = True, dropout = 0.3, bidirectional = True),
#            nn.Dropout(0.3)
            nn.Linear(in_features = embedding_dim , out_features = embedding_dim, bias = True , device = None , dtype = None ),
            nn.Softmax(dim = None)
#            self.model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        )
        
    
        
    def load_article(self, article, vectorized = 1, labels = 0):
        self.article = tokenize_article(article, vectorized, labels)
        self.max_sentence_length = lambda max_sentence_length: max(article.map(len))
#        print(tokenize_article(article, vectorized, labels))
        return self.article
    
    def load_all_articles(self, dataset, vectorized = 1):
        training_dataset = []
        labels_dataset = []
        for article in dataset:
#            print(article)
#            print(self.load_article(article, vectorized = 1, labels = 0))
            training_dataset.append(self.load_article(article, vectorized = 1, labels = 0))
            labels_dataset.append(self.load_article(article, vectorized = 1, labels = 1))
        #print(training_dataset)
        return training_dataset, labels_dataset
    
    def forward(self, dataset, epochs = 5):
        training_dataset = torch.tensor(self.training_dataset)
        labels_dataset = torch.tensor(self.labels_dataset)
        loss_function = nn.CrossEntropyLoss
        optmimizer = torch.optim.Adam(self.parameters(), self.lr)
        for epoch in range(epochs):
            loss = loss_function(training_dataset, labels_dataset)
            loss.backwards()
            optimizer.step()
            print(loss)
        #self.modnlp(training_dataset, labels_dataset)
        
#        print(self.model.summary())

In [ ]:
text_generator = RNN_LSTM(default_length, hidden_dim = 50, n_layers = 2, lr = 0.003).forward(dataset)

In [ ]:
#if nlp_df:
#    del nlp_df